# Not in this notebook
- computing the CAPPI layer (Takes too long)


# In this notebook
- Processing all data types and leaving the nan values so they can be removed at the latest possible moment


# Imports

In [1]:
import numpy as np
import pandas as pd
import json
import datetime
from dataProcessingHelpers import *
from missingno import matrix

min_date = datetime.datetime(2015,1,1)
URL_save_data = 'C:/Users/sverrirhd/Google Drive/Skóli/DTU/Thesis/Programming/Experiments/Data/'

URL_raw_temperature_data = 'C:/Users/sverrirhd/Google Drive/Skóli/DTU/Thesis/Data/Temperature data/hbs_hitastig_klst.csv'


URL_processed_temperature_data = URL_save_data + 'temperature_data.pkl'
URL_processed_temperature_forecast = URL_save_data + 'temperature_forecast_data.pkl'
URL_processed_drainage_flow = URL_save_data + 'drainage_flow.pkl'


# Temperature data
- Read raw data
- Remove bad values
- Pivot table
- save data

In [2]:
df_temp_long = pd.read_csv(URL_raw_temperature_data,sep='\t')
df_temp_long.loc[:,'TIMI'] = pd.to_datetime(df_temp_long.loc[:,'TIMI'])
df_temp_long.loc[:,'T'] = df_temp_long.loc[:,'T'].replace({'(null)':np.nan}).astype(float)
df_temp_long.STOD = df_temp_long.STOD.astype('str')
df_temp = df_temp_long.pivot_table(values='T',index='TIMI',columns='STOD')
df_temp = df_temp.loc[min_date:].astype(float)
df_temp.to_pickle(URL_processed_temperature_data)

# Temperature forecast
- Requires a VPN connection with Veitur
- Queries all historical forecasts for weather station 1475
- Processes data:
    - Datetime conversions
    - Computes the offset of the prediction vs the predicted date
    - Pivots the table so that each row is 'now' and each consecutive column represents +1 hour into the future
    - Resamples data to hourly intervals and replaces missing data with np.nan
    - Interpolates backwards (i.e. won't try to extrapolate beyond last available forecast date)
    - Fill in missing spaces with best available forecast. (i.e. newest forecast that has predicted for that period)

In [3]:
query_temp_forecast =\
''' SELECT date_time,value_date,temperature
    FROM [DM_Orkuveitan].[dbo].[fact_vedurspar]
    WHERE station_id = 1475 
    AND value_date >= \'2014-12-20\' 
    AND value_date <= \'2021-01-01\'
'''
df_spa_raw = execute_query(connect_to_db(),query_temp_forecast)
df_spa = process_temperature_predictions(df_spa_raw,min_date).astype(float)
df_spa.to_pickle(URL_processed_temperature_forecast)

# Drainage data
- Open metadata file
- Get list of sensor names to use
- Load each batch of sensor dataframes into a list 
- Use only average values (ctype = 4) with a good flag (flag = 0)
- Pivot all tables into a single dataframe with a common datetime index
- If any value is less than 0 or equal to 0, 1 or 2 precisely, then they are errors, so we remove those

In [4]:
f = open(URL_save_data + 'drainage_meta_data.json', encoding='utf-8')
df = pd.DataFrame(json.load(f)['sensors'])

get_flow = lambda x : sensor_data_query(x['ID'], x['flow'],earliest_date = min_date)
df.loc[:,'flow_query'] = df.apply(get_flow,axis=1)

conn = connect_to_db()
list_flow = []
for index,row in df.iterrows():
    data_flow_raw = execute_query(conn,row['flow_query'],index_col = 'CTime')
    data_flow = process_drainage_sensors(data_flow_raw)
    data_flow.name = row['ID']
    list_flow.append(data_flow)
    
df_drainage_flow = pd.concat(list_flow,axis=1).astype(float)
df_drainage_flow.to_pickle(URL_processed_drainage_flow)

In [ ]:
.reshape(len(df_X_NWP),-1,10,10)[:,:pred_dist]